In [2]:
import pandas as pd
from geocoding import geocode_unique_addresses

In [2]:
#Import raw tiltrisk data from Bertrand
data = pd.read_csv('tiltrisk_alpha.csv')
#Drop empty columns
data_onlyuseful = data.dropna(axis=1)
#Read full tilt data from Tilman
fulldata = pd.read_csv("webtool_at_company_level_05_09_24.csv")
#Merge with Bertrands data to get addresses in the same dataframe
data_withaddress = data_onlyuseful.merge(
    fulldata[['companies_id', 'address']].drop_duplicates().reset_index(drop=True),
    on='companies_id',
    how='left'
)
#Save as a Feather file
data_withaddress.to_feather("tiltrisk.feather")

In [3]:
data_withaddress = pd.read_feather("tiltrisk.feather")

In [ ]:
#Run the geocoding
geocode_unique_addresses(data_withaddress,"geocoded_tiltrisk.csv")

In [10]:
geocoded_data = pd.read_csv('geocoded_tiltrisk.csv')
data_localised = data_withaddress.merge(geocoded_data,how='left',on='address')

In [12]:
data_localised.to_feather('tiltrisk_geocoded.feather')

# Remove duplicates and pivot data

In [62]:
geodf_deduped = data_localised.drop_duplicates()
# Step 1: Get columns that will be pivoted based on 'term'
value_columns = ['net_present_value_baseline', 'net_present_value_shock', 'pd_baseline',
       'pd_shock', 'net_present_value_difference',
       'crispy_perc_value_change', 'pd_difference']

# Step 2: Pivot the DataFrame
data_pivoted = geodf_deduped.pivot(index=geodf_deduped.columns.difference(value_columns).drop('term'), columns='term', values=value_columns)

# Step 3: Flatten the multi-level columns generated by pivoting (optional but makes the output more readable)
data_pivoted.columns = [f"{col[1]}_{col[0]}" for col in data_pivoted.columns]

# Step 4: Reset index to make `shock_scenario` a column again if needed
data_pivoted = data_pivoted.reset_index()
# Now `data_pivoted` is your pivoted DataFrame with columns spread by 'term' and index as 'shock_scenario'
data_pivoted_meaningful = data_pivoted.dropna(axis=1)


In [64]:
data_pivoted_meaningful.to_feather("tiltrisk_pivoted.feather")

In [86]:
import folium
from folium.plugins import HeatMapWithTime, HeatMap

# Selection for weight column to visualize
weight = 'pd_shock'

# Select baseline scenario
baseline_scenario = 'IPR2023_baseline'

# Select shock scenario based on valid options from filtered data
shock_scenario = 'IPR2023_FPS'

sector = 'Coal'

# Filter data to include only rows with valid latitude, longitude, and selected weight
data_withaddress = data_localised.loc[
    (data_localised['baseline_scenario'] == baseline_scenario) &
    (data_localised['shock_scenario'] == shock_scenario)
].dropna(subset=['latitude', 'longitude', 'term',weight]).copy()

# Group data by 'term' and create a list of heatmap data for each time point
# Each entry in the list corresponds to the data for a particular 'term' time period
heat_data = [
    data_withaddress[data_withaddress['term'] == t][['latitude', 'longitude', weight]].values.tolist()
    for t in sorted(data_withaddress['term'].unique())
]


# Create the base map
m = folium.Map(
    location=[data_withaddress['latitude'].mean(), data_withaddress['longitude'].mean()], 
    zoom_start=8
)
# Add a heatmap layer with time support
HeatMapWithTime(
    heat_data
).add_to(m)

m

In [97]:
import folium
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

# Load the NUTS shapefile
nuts_gdf = gpd.read_file("NUTS_RG_60M_2024_4326.shp")
nuts_gdf = nuts_gdf[nuts_gdf['LEVL_CODE'] == 3]

print(nuts_gdf.columns)

# Filter your data as before
baseline_scenario = 'IPR2023_baseline'
shock_scenario = 'IPR2023_FPS'
weight = 'pd_shock'

# Filter data for the selected scenarios and drop rows with missing coordinates or weights
data_withaddress = data_localised.loc[
    (data_localised['baseline_scenario'] == baseline_scenario) & 
    (data_localised['shock_scenario'] == shock_scenario)
].dropna(subset=['latitude', 'longitude', 'term', weight]).copy()

# Convert your data into a GeoDataFrame
data_withaddress['geometry'] = data_withaddress.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
data_gdf = gpd.GeoDataFrame(data_withaddress, geometry='geometry', crs="EPSG:4326")

# Spatially join the data with the NUTS boundaries
# This will add the NUTS region information to each data point
data_with_nuts = gpd.sjoin(data_gdf, nuts_gdf, how="left", predicate="within")

# Aggregate the `pd_shock` by NUTS region (using the NUTS region identifier, e.g., 'NUTS_ID')
aggregated_data = data_with_nuts.groupby('NUTS_ID')[weight].sum().reset_index()

# Merge the aggregated data back with the NUTS shapefile for mapping
nuts_gdf = nuts_gdf.merge(aggregated_data, on='NUTS_ID', how='left')

# Initialize the base map centered on the data points
m = folium.Map(
    location=[data_withaddress['latitude'].mean(), data_withaddress['longitude'].mean()],
    zoom_start=8
)

# Add a choropleth layer based on the NUTS boundaries
folium.Choropleth(
    geo_data=nuts_gdf,
    name='choropleth',
    data=nuts_gdf,
    columns=['NUTS_ID', weight],
    key_on='feature.properties.NUTS_ID',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='PD Shock Intensity by Region'
).add_to(m)

# Add layer control and display the map
folium.LayerControl().add_to(m)
m


Index(['NUTS_ID', 'LEVL_CODE', 'CNTR_CODE', 'NAME_LATN', 'NUTS_NAME',
       'MOUNT_TYPE', 'URBN_TYPE', 'COAST_TYPE', 'geometry'],
      dtype='object')
